In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

今回のコンペでは、与えられた気象データから、carbon monoxide（一酸化炭素）、benzene（ベンゼン）、nitrogen oxides（窒素酸化物）の3つの量を予測する。

In [ ]:
import matplotlib.pyplot as plt 
import seaborn as sns
import sklearn
import xgboost as xgb
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
plt.rcParams["font.size"] = 18

学習用、テスト用、提出用データをそれぞれ読み込む

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jul-2021/train.csv')
test = pd.read_csv("../input/tabular-playground-series-jul-2021/test.csv")
sample_submission = pd.read_csv("../input/tabular-playground-series-jul-2021/sample_submission.csv")

## データの確認

In [ ]:
# 学習用データ（特徴量と目的変数）
train

In [ ]:
# テスト用データ（特徴量）
test

In [ ]:
# 提出用データ（目的変数には適当な値が入っている）
sample_submission

In [ ]:
train.info()
print('=' * 70)
test.info()

## データの前処理


特徴量の一つ、date_timeをstr型（文字列型）からdatetime型に変換する

In [ ]:
train['date_time'] = pd.to_datetime(train['date_time'], format = "%Y-%m-%d %H:%M:%S")
test['date_time'] = pd.to_datetime(test['date_time'], format= "%Y-%m-%d %H:%M:%S")
target_name = train.columns[-3:].values # 目的変数名を保存しておく
target_name

与えられた特徴量から、新たにさまざまな特徴量を追加する

In [ ]:
def makeFeatures(df):
    # 温度のn日前との差分
    df["temp-3"] = df["deg_C"] - df["deg_C"].shift(periods=3, fill_value=0)
    df["temp-6"] = df["deg_C"] - df["deg_C"].shift(periods=6, fill_value=0)
    df["temp-24"] = df["deg_C"] - df["deg_C"].shift(periods=24, fill_value=0)
    
    # 絶対湿度のn日前との差分
    df["abs-3"] = df["absolute_humidity"] - df["absolute_humidity"].shift(periods=3, fill_value=0)
    df["abs-6"] = df["absolute_humidity"] - df["absolute_humidity"].shift(periods=6, fill_value=0)
    df["abs-24"] = df["absolute_humidity"] - df["absolute_humidity"].shift(periods=24, fill_value=0)
    
    # 相対湿度のn日前との差分
    df["rel-3"] = df["relative_humidity"] - df["relative_humidity"].shift(periods=3, fill_value=0)
    df["rel-6"] = df["relative_humidity"] - df["relative_humidity"].shift(periods=6, fill_value=0)
    df["rel-24"] = df["relative_humidity"] - df["relative_humidity"].shift(periods=24, fill_value=0)
    
    # センサのn日前との差分
    df["s1-3"] = df["sensor_1"] - df["sensor_1"].shift(periods=3, fill_value=0)
    df["s2-3"] = df["sensor_2"] - df["sensor_2"].shift(periods=3, fill_value=0)
    df["s3-3"] = df["sensor_3"] - df["sensor_3"].shift(periods=3, fill_value=0)
    df["s4-3"] = df["sensor_4"] - df["sensor_4"].shift(periods=3, fill_value=0)
    df["s5-3"] = df["sensor_5"] - df["sensor_5"].shift(periods=3, fill_value=0)
    df["s1-6"] = df["sensor_1"] - df["sensor_1"].shift(periods=6, fill_value=0)
    df["s2-6"] = df["sensor_2"] - df["sensor_2"].shift(periods=6, fill_value=0)
    df["s3-6"] = df["sensor_3"] - df["sensor_3"].shift(periods=6, fill_value=0)
    df["s4-6"] = df["sensor_4"] - df["sensor_4"].shift(periods=6, fill_value=0)
    df["s5-6"] = df["sensor_5"] - df["sensor_5"].shift(periods=6, fill_value=0)
    df["s1-24"] = df["sensor_1"] - df["sensor_1"].shift(periods=24, fill_value=0)
    df["s2-24"] = df["sensor_2"] - df["sensor_2"].shift(periods=24, fill_value=0)
    df["s3-24"] = df["sensor_3"] - df["sensor_3"].shift(periods=24, fill_value=0)
    df["s4-24"] = df["sensor_4"] - df["sensor_4"].shift(periods=24, fill_value=0)
    df["s5-24"] = df["sensor_5"] - df["sensor_5"].shift(periods=24, fill_value=0)
    
    # センサ同士の差分
    df['S1-S2']=df['sensor_1']-df['sensor_2']
    df['S1-S3']=df['sensor_1']-df['sensor_3']
    df['S1-S4']=df['sensor_1']-df['sensor_4']
    df['S1-S5']=df['sensor_1']-df['sensor_5']
    df['S2-S3']=df['sensor_2']-df['sensor_3']
    df['S2-S4']=df['sensor_2']-df['sensor_4']
    df['S2-S5']=df['sensor_2']-df['sensor_5']
    df['S3-S4']=df['sensor_3']-df['sensor_4']
    df['S3-S5']=df['sensor_3']-df['sensor_5']
    df['S4-S5']=df['sensor_4']-df['sensor_5']
    
    df["SMC"] = (df["absolute_humidity"] * 100) / df["relative_humidity"]
    
    # 時間に関する特徴量
    df["month"] = df["date_time"].dt.month # 月
    df["day_of_week"] = df["date_time"].dt.dayofweek # 曜日（月曜始まり）
    df["day_of_year"] = df["date_time"].dt.dayofyear # 1月1日から数えて何日目か
    df["hour"] = df["date_time"].dt.hour # 時
    df["quarter"] = df["date_time"].dt.quarter # 四半期
    df["week_of_year"] = df["date_time"].dt.isocalendar().week.astype(int) # 1月1日から数えて何週目か
    df["is_sprint"] = df["month"].isin([3, 4, 5]).astype(int) # 春
    df["is_summer"] = df["month"].isin([6, 7, 8]).astype(int) # 夏
    df["is_autumn"] = df["month"].isin([9, 10, 11]).astype(int) # 秋
    df["is_winter"] = df["month"].isin([1, 2, 12]).astype(int) # 冬
    df["working_hours"] =  df["hour"].isin(np.arange(8, 19, 1)).astype(int) # 勤務時間(8時～19時)
    df["is_weekend"] = (train["date_time"].dt.dayofweek >= 5).astype(int) # 週末
    return df

In [ ]:
train = makeFeatures(train)
test = makeFeatures(test)

datetime型の特徴量を学習に用いるため、int型（整数型）に変換する

In [ ]:
train['date_time'] = train['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9
test['date_time'] = test['date_time'].astype('datetime64[ns]').astype(np.int64)/10**9

学習用データの中身は3つの項目（target_carbon_monoxide, target_benzene, target_nitrogen_oxides）が目的変数となっていて、のこりが特徴量（説明変数）となっている。  
また、テスト用データの中身は学習用データから3つの目的変数を除いた構造になっている。  
以下のプログラムでは、

* 学習用データから特徴量だけを取り出したデータ（X_train）
* 学習用データから目的変数だけ取り出したデータ（target）
* テスト用データ（X_test）

を作成している。

In [ ]:
columns = test.columns
X_train = train[columns].values

target0 = train['target_carbon_monoxide'].values.reshape(-1,1)
target1 = train['target_benzene'].values.reshape(-1,1)
target2 = train['target_nitrogen_oxides'].values.reshape(-1,1)
target = np.concatenate([target0, target1, target2], 1)

X_test = test

それぞれの特徴量によって、スケールが全く違うので標準化によって平均0、分散1に変換する。  
($\mu$：平均, $\sigma$：標準偏差)
$$z =  \frac{x - \mu}{\sigma}$$

In [ ]:
stdscaler = StandardScaler()
X_train_transed = stdscaler.fit_transform(X_train)
X_test_transed = stdscaler.transform(X_test)

## 前処理後のデータ確認

In [ ]:
# 学習用データ（特徴量）
X_train_transed_df = pd.DataFrame(data=X_train_transed, columns=columns)
X_train_transed_df

In [ ]:
# テスト用データ（特徴量）
X_test_transed_df = pd.DataFrame(data=X_test_transed, columns=columns)
X_test_transed_df

In [ ]:
# 目的変数
target_df = pd.DataFrame(data=target, columns=target_name)
target_df

In [ ]:
train.info()
print('=' * 50)
test.info()

In [ ]:
# 学習用データの内容を表示する
print(f"学習用データ：{train.shape[0]}行{train.shape[1]}列")
print(f"学習用データ数：{train.shape[0]}個")
print()
print(f"特徴量のデータ：{X_train_transed.shape[0]}行{X_train_transed.shape[1]}列")
print(f"特徴量の数：{X_train_transed.shape[1]}個")
print(f"特徴量の名前：{columns.values}")
print()
print(f"目的変数のデータ：{target.shape[0]}行{target.shape[1]}列")
print(f"目的変数の数：{target.shape[1]}個")
print(f"目的変数の名前：{target_name}")

In [ ]:
# テスト用データの内容を表示する
print(f"テスト用データ：{test.shape[0]}行{test.shape[1]}列")
print(f"テスト用データ数：{test.shape[0]}個")
print()
print(f"特徴データ：{X_test_transed.shape[0]}行{X_test_transed.shape[1]}列")
print(f"特徴量の数：{X_test_transed.shape[1]}個")
print(f"特徴量の種類：{columns.values}")

In [ ]:
# 提出用データの内容を表示する
print(f"提出用データ：{sample_submission.shape[0]}行{sample_submission.shape[1]}列")
print(f"提出用データ数：{sample_submission.shape[0]}個")

In [ ]:
"""feature_num = X_train_transed.shape[1]
nrows = feature_num // 2 if feature_num % 2 == 0 else feature_num // 2 + 1

plt.figure(figsize=(16, nrows * 4))
plt.subplots_adjust(hspace=0.4)
values = columns.values
for i, col in enumerate(values, start=1):
    plt.subplot(nrows, 2, i)
    plt.hist(train[col], bins=50)
    plt.title(col)"""

In [ ]:
"""feature_num = X_test_transed.shape[1]
nrows = feature_num // 2 if feature_num % 2 == 0 else feature_num // 2 + 1

plt.figure(figsize=(16, nrows * 4))
plt.subplots_adjust(hspace=0.4)
values = columns.values
for i, col in enumerate(values, start=1):
    plt.subplot(nrows, 2, i)
    plt.hist(test[col], bins=50)
    plt.title(col)"""

特徴量毎の分布を調べるため、ヒストグラムを生成する。  
学習用データを青色（濃い）、テスト用データをオレンジ（薄い）で表している。

In [ ]:
feature_num = X_train_transed.shape[1]
nrows = feature_num // 2 if feature_num % 2 == 0 else feature_num // 2 + 1

plt.figure(figsize=(16, nrows * 4))
plt.subplots_adjust(hspace=0.4)
values = columns.values
for i, col in enumerate(values, start=1):
    plt.subplot(nrows, 2, i)
    plt.hist(train[col], bins=50, density=True)
    plt.hist(test[col], bins=50, alpha=0.7, density=True)
    plt.title(col)

目的変数の分布を調べるため、ヒストグラムを生成する。

In [ ]:
feature_num = X_test_transed.shape[1]
nrows = len(target_name)

# グラフを用意
plt.figure(figsize=(16, 12))
plt.subplots_adjust(hspace=0.4)
for i, col in enumerate(target_name, start=1):
    plt.subplot(nrows, 1, i)
    plt.hist(target_df[col], bins=50)
    plt.title(col)

各特徴量、目的変数間の相関関係を調べるために、ヒートマップを使って表示する。  
白色に近いほど正や負の相関関係が強く、黒色に近いほど相関関係が弱い

In [ ]:
plt.figure(figsize=(18, 18)) 
sns.heatmap(pd.concat([X_train_transed_df, target_df], axis=1).corr(), square=True, vmax=1, vmin=-1, center=0, cbar_kws={"shrink": 0.82})

## 勾配ブースティング（XGboost）による学習

XGboostのモデルを構築する。ただし、目的変数が3つあるため3つのモデルを生成する。

In [ ]:
boosting = [0] * 3
for i in range(3):
    boosting[i] = xgb.XGBRegressor(n_estimators = 50,
                                   eta = 0.1,
                                   max_depth = 5, 
                                   random_state = 1,
                                   subsample = 0.8
                                  )

XGboostのハイパーパラメータを決めるため、グリッドサーチを行い、最適なパラメータを選択する。また、5分割交差検証を行うことで、過学習を防ぎ正しく精度を評価する。

In [ ]:
def print_results(results):
    print('BEST PARAMS: {}\n'.format(results.best_params_))
    rank = results.cv_results_['rank_test_score']
    arg = np.argsort(rank)
    rank = np.sort(rank)
    means = results.cv_results_['mean_test_score'][arg]
    stds = results.cv_results_['std_test_score'][arg]
    params = np.array(results.cv_results_['params'])[arg]

    for r, mean, std, param in zip(rank, means, stds, params):
        print('No{} : {} (+/-{}) for {}'.format(r, round(mean, 3), round(std * 2, 3), param))

def gridSearch(model, X_train, y_train):
    paramters = {
        'n_estimators': [50, 100],
        'max_depth':[4], 
        'learning_rate': [0.3],
        'gamma': [0.1], 
        'min_child_weight': [0.8, 1, 1.2], 
        'subsample': [0.8], 
        'colsample_bytree': [0.8],
    }
    
    grid = GridSearchCV(
        estimator = model,
        param_grid = paramters,
        scoring = 'neg_root_mean_squared_error',
        cv = 5,
        verbose = 2
    )
    grid.fit(X_train, y_train)
    print_results(grid)

In [ ]:
#gridSearch(boosting[0], X_train_transed, target[:, 0])

実際にXGboostで学習する。

In [ ]:
for i, boost in enumerate(boosting):
    y = target[:, i]
    boost.fit(X_train_transed, y)

## 学習結果

学習したXGboostモデルを使用して、テストデータから予測する。

In [ ]:
for i, boost in enumerate(boosting):
    pred = boost.predict(X_test_transed)
    sample_submission[sample_submission.columns[i + 1]] = pred
sample_submission.head()

予測した結果をファイルに保存する

In [ ]:
sample_submission.to_csv('submission.csv', index=False)

テスト用データは目的変数が公開されていないため、学習用データを使って予測結果をグラフに表示する。

In [ ]:
for i, boost in enumerate(boosting):
    print(f'{target_name[i]} = {boost.score(X_train_transed, target[:, i])}')

In [ ]:
plt.figure(figsize=(15, 20))
for i, name in enumerate(target_name):
    plt.subplot(3, 1, i + 1)
    plt.plot(target[2000:3000, i], label='True')
    plt.plot(boosting[i].predict(X_train_transed)[2000:3000], label='Predict')
    plt.title(name)
    plt.legend(loc='upper right')